# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from z3 import *
import sys
sys.path.append('../../scripts/biota/')
from BIoTA_Control import *
from BIoTA_Attack import *
import warnings
warnings.filterwarnings('ignore')

# Fixed Parameter

In [2]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)
MINUTES_IN_A_DAY = 1440         # Number of minutes in a day
OFF_PEAK_ENERGY_COST = 0.34     # OFF-PEAK energy cost (USD)
ON_PEAK_ENERGY_COST = 0.48      # ON-PEAK energy cost (USD)
ON_PEAK_START_SLOT = 960        # ON-PEAK start time (minute in a day)
ON_PEAK_END_SLOT = 1260         # ON-PEAK end time (minute in a day)
BATTER_STORAGE = 0.48           # Energy (kWh) produced by battery

# Variable Parameters

In [3]:
zone_temp_setpoint = [0, 75.2, 75.2, 75.2, 75.2]     # list of temperature (fahrenheit) setpoint of the different zones
zone_co2_setpoint = [0, 1000, 1000, 1000, 1000]      # list of CO2 (ppm) setpoint of the corresponding zones
control_time = 1                                     # control time (in minute)

# Accessing Zone-Activity Information

In [4]:
zones = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                  # Zones' volumes (cubic feet)
pp_co2 = zones["CO2 Emission by Occupant (cfm)"].to_list()    # CO2 Emission by Occupant (cfm)
pp_heat = zones["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
load = zones["Heat Radiated by Appliances (W)"].to_list()     # Heat radiated by Appliances (W)
        
activities = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Activity-Info')

activity_zone_map = dict()
for i in range(len(activities)):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

# Generating Possible Combination of Occupants

In [5]:
import itertools
all_samples = list(itertools.permutations([1, 1, 0, 0, 0]))
all_samples += list(itertools.permutations([2, 0, 0, 0, 0]))
# however, there are repetations

unique_samples = set()

for data in all_samples:    
    unique_samples.add(data)
    
unique_samples

{(0, 0, 0, 0, 2),
 (0, 0, 0, 1, 1),
 (0, 0, 0, 2, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 2, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 2, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (2, 0, 0, 0, 0)}

# Tabulate the Control Costs for all Possible Combinations

In [6]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_control_cost[sample] = control_cost(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)
dict_control_cost

{(1, 0, 1, 0, 0): 0.006996611666666667,
 (1, 1, 0, 0, 0): 0.006524535000000001,
 (0, 0, 1, 0, 1): 0.036949184833333336,
 (2, 0, 0, 0, 0): 0.0,
 (0, 0, 0, 2, 0): 0.062190949333333335,
 (0, 0, 0, 0, 2): 0.03490514633333333,
 (0, 1, 0, 0, 1): 0.03647710816666667,
 (1, 0, 0, 0, 1): 0.029952573166666666,
 (0, 0, 1, 1, 0): 0.06309208633333334,
 (0, 2, 0, 0, 0): 0.012034600000000001,
 (0, 0, 0, 1, 1): 0.08604804783333334,
 (0, 1, 0, 1, 0): 0.06262000966666667,
 (0, 0, 2, 0, 0): 0.009909890000000001,
 (0, 1, 1, 0, 0): 0.013521146666666668,
 (1, 0, 0, 1, 0): 0.056095474666666666}

# Tabulate the Attack Vectors for all Possible Combinations

In [7]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_attack_vectors = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_attack_vectors[sample] = attack_vector(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)
dict_attack_vectors

{(1, 0, 1, 0, 0): ([1, 0, 0, 1, 0],
  [0, 0, -1, 1, 0],
  [0.0, 0.0, -17.876412755463104, -16.526147257402087, 0.0],
  [0, 0, 4.513888888889028, -22.805555555555543, 0]),
 (1, 1, 0, 0, 0): ([1, 0, 0, 1, 0],
  [0, -1, 0, 1, 0],
  [0.0, -17.85909606660449, 0.0, -16.526147257402087, 0.0],
  [0, 7.330246913580254, 0, -22.805555555555543, 0]),
 (0, 0, 1, 0, 1): ([0, 0, 0, 2, 0],
  [0, 0, -1, 2, -1],
  [0.0, 0.0, -17.876412755463104, -15.274516737026401, -18.6343599816567],
  [0, 0, 4.513888888889028, -45.611111111111086, 12.789351851851848]),
 (2, 0, 0, 0, 0): ([2, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0.0, 0.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0]),
 (0, 0, 0, 2, 0): ([0, 0, 0, 2, 0],
  [0, 0, 0, 0, 0],
  [0.0, 0.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0]),
 (0, 0, 0, 0, 2): ([0, 0, 0, 2, 0],
  [0, 0, 0, 2, -2],
  [0.0, 0.0, 0.0, -15.274516737026401, -19.49094218553562],
  [0, 0, 0, -45.611111111111086, 25.578703703703695]),
 (0, 1, 0, 0, 1): ([0, 0, 0, 2, 0],
  [0, -1, 0, 2, -1],
  [0.0, -17.85909606

In [8]:
def biota_attack(house, filename):
    # reading processed dataset
    control_costs = []
    attack_costs = []
    unit_energy_costs = []
    energy_consumptions = []
    attack_energy_consumptions = []
    
    attack_dataset = pd.DataFrame(columns = ['Day', 'Minute', 'Zone Occupant (Outdoor) (person)', 'Zone Occupant (Bedroom) (person)', 'Zone Occupant (Livingroom) (person)', 'Zone Occupant (Kitchen) (person)' ,'Zone Occupant (Bathroom) (person)', 'Attacked Zone Occupant (Outdoor) (person)', 'Attacked Zone Occupant (Bedroom) (person)', 'Attacked Zone Occupant (Livingroom) (person)', 'Attacked Zone Occupant (Kitchen) (person)' ,'Attacked Zone Occupant (Bathroom) (person)', 'Required Injection in Occupancy Sensor Measurement (Outdoor) (person)', 'Required Injection in Occupancy Sensor Measurement (Bedroom) (person)', 'Required Injection in Occupancy Sensor Measurement (Livingroom) (person)', 'Required Injection in Occupancy Sensor Measurement (Kitchen) (person)', 'Required Injection in Occupancy Sensor Measurement (Bathroom) (person)', 'Required Injection in Temperature Sensor Measurement (Outdoor) (degree F)', 'Required Injection in Temperature Sensor Measurement (Bedroom) (degree F)', 'Required Injection in Temperature Sensor Measurement (Livingroom) (degree F)', 'Required Injection in Temperature Sensor Measurement (Kitchen) (degree F)', 'Required Injection in Temperature Sensor Measurement (Bathroom) (degree F)', 'Required Injection in CO2 Sensor Measurement (Outdoor) (ppm)', 'Required Injection in CO2 Sensor Measurement (Bedroom) (ppm)', 'Required Injection in CO2 Sensor Measurement (Livingroom) (ppm)', 'Required Injection in CO2 Sensor Measurement (Kitchen) (ppm)', 'Required Injection in CO2 Sensor Measurement (Bathroom) (ppm)', 'Benign Control Cost ($)', 'Attack Control Cost ($)'])
    control_dataset = pd.read_csv('../../data/processed/Processed-Dataframe_House-' + house + '_BIoTA.csv')

    prev_day = -1
    battery_usage = 0 # in kWh
    
    for i in range(len(control_dataset)):
        
        current_day = int(control_dataset.iloc[i,0])
        current_minute = int(control_dataset.iloc[i,1])
        
        if current_day!= prev_day:
            battery_usage = 0
            prev_day = current_day        
            print(house, current_day)
            
        zone_occupants = to_tuple(control_dataset.iloc[i][2:].values)
        attack_info = dict_attack_vectors[zone_occupants]
        attack_zone_occupants = attack_info[0]
        del_zone_occ = attack_info[1]
        del_zone_temp = attack_info[2]
        del_zone_co2 = attack_info[3]
        
        energy_consumption = dict_control_cost[zone_occupants]
        attack_energy_consumption = dict_control_cost[to_tuple(attack_zone_occupants)]
        
        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST
                
        benign_cost = energy_consumption * unit_energy_cost
        attack_cost = attack_energy_consumption * unit_energy_cost

    
        record = [[current_day, current_minute] + list(zone_occupants) + attack_zone_occupants + del_zone_occ + del_zone_temp + del_zone_co2 + [benign_cost, attack_cost]]
        attack_dataset = attack_dataset.append(pd.DataFrame(record, columns=attack_dataset.columns), ignore_index=True)
    
    attack_dataset.to_csv(filename, index = False)

# Saving Datasets

In [9]:
biota_attack("A", "../../data/biota/BIoTA-Attack-Dataframe_House-A.csv")
biota_attack("B", "../../data/biota/BIoTA-Attack-Dataframe_House-B.csv")

A 1
A 2
A 3
A 4
A 5
A 6
A 7
A 8
A 9
A 10
A 11
A 12
A 13
A 14
A 15
A 16
A 17
A 18
A 19
A 20
A 21
A 22
A 23
A 24
A 25
A 26
A 27
A 28
A 29
A 30
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
B 27
B 28
B 29
B 30
